# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121659e+02     1.521084e+00
 * time: 0.08636903762817383
     1     1.129413e+01     1.026594e+00
 * time: 0.9283490180969238
     2    -1.159892e+01     1.000282e+00
 * time: 1.167492151260376
     3    -3.387364e+01     9.132988e-01
 * time: 1.3341541290283203
     4    -4.731970e+01     7.032257e-01
 * time: 1.7933390140533447
     5    -5.652348e+01     3.062603e-01
 * time: 2.139111042022705
     6    -5.950574e+01     2.011360e-01
 * time: 2.2486321926116943
     7    -6.076972e+01     7.672595e-02
 * time: 2.357631206512451
     8    -6.131474e+01     5.486218e-02
 * time: 2.457515001296997
     9    -6.161023e+01     5.409149e-02
 * time: 2.5648281574249268
    10    -6.180991e+01     3.396100e-02
 * time: 2.6717731952667236
    11    -6.196945e+01     2.274668e-02
 * time: 2.7780821323394775
    12    -6.206093e+01     1.781598e-02
 * time: 2.8781561851501465
    13    -6.210244e+01     1.586214e-02
 * time: 2.9849531650543

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671072
    AtomicLocal         -18.8557688
    AtomicNonlocal      14.8522647
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485378 
    Xc                  -19.3336823

    total               -62.261666460510
